# Retrieving substation's connected power stations in French transmission network

RTE does not supply geo-referenced data for power stations located in the French territory. However, it does come with EIC as well as INSEE, IRIS and CodNat codes from the French codification system. These are the two available methods for retrieving location of the French power stations accordingly:
* With the IRIS code and the corresponding IRIS slicing of the territory provided by IGN, it is possible to get the approximate location of the given power facilities.
* With the infrastructure's name, the city/commune's name and the INSEE code, we may use some geocoding API through Python libraries [geopy](https://github.com/geopy/geopy) or [geocoder](https://github.com/DenisCarriere/geocoder), to get the exact address. However, accuracy is far from guaranteed, as many of the biggest power plants are generally not recorded in the corresponding databases.

### List of power stations connected to substations
We see that retrieving exact location of French power facilities is not guaranteed. Rather, with the CodNat of both substations and power stations, as well as the IRIS code, it is possible to retrieve *to which substation a given facility belongs (i.e. is connected)*.

Hence, we propose an approach where location of the power stations is not stored, but instead each substation in the network will have its own list of connected power facilities, each one being characterized by its power, voltage, etc.

### Creating *virtual* substations
It happens that the power plant is directly connected to another substation far from the production site, especially in the case of nuclear power. In that case, we do not know to which substation the power plant is connected, since CodNat as well as IRIS codes are not shared between substations and power facilities. It is therefore necessary to create some *virtual* substation hosting the given power plant connection.

***

## Retrieve power stations connected to substations

In [1]:
import pandas as pd
import os
from greece import data_dir
from greece.gistools import PolygonLayer
from greece.pfmtools.grid import PowerLine, Substation
from greece.pfmtools.tools import rte_substations_with_power_plants

First import RTE listing of power stations and format it, that is add __voltage__ and __codnat__ keys, format iris code as a string (as in iris layer) + create a key __power__ which is either the nominal installed capacity or the max connected power (in case installed capacity is equal to 0):

In [2]:
power_stations = pd.read_csv(os.path.join(data_dir, "france_rte/list_power_stations.csv"), dtype=object)
power_stations["voltage"] = power_stations["Tension raccordement"][~power_stations[
    "Tension raccordement"].isna()].apply(lambda x: x.replace(" ", ""))
power_stations = power_stations.rename(index=str, columns={"codNat": "codnat"})
power_stations["power"] = power_stations["puisMaxInstallee"].combine(power_stations["puisMaxRac"],
                                                                     lambda x, y: x if x != 0 else y)
power_stations["CODE_IRIS"] = power_stations["CODE_IRIS"][~power_stations["CODE_IRIS"].isna()].apply(
    lambda x: str(int(x)))

Then import IRIS slicing of the territory as well as RTE substations and power lines:

In [3]:
substations = Substation(os.path.join(data_dir, "france_rte/substations/substations.shp"), "tensionmax")
substations = substations.rename("tensionmax", "voltage")
pwlines = PowerLine(os.path.join(data_dir, "france_rte/france_pwlines/france_pwlines.shp"), "tension")
pwlines = pwlines.rename("tension", "voltage")
iris = PolygonLayer(os.path.join(data_dir, "iris/iris.shp"))

We are only interested in power stations located in Metropolitan France (remove Corsica and overseas regions) and in operation (i.e. _en service_ in french):

In [4]:
power_stations = power_stations[power_stations["CODE_IRIS"].isin(iris["CODE_IRIS"])]
power_stations = power_stations[~power_stations["codnat"].isna()]
power_stations = power_stations[power_stations["regime"] == "En service"]

Retrieve the whole set of substations with _real_ and _virtual_ ones, with corresponding connect power stations:

In [5]:
france_substations = rte_substations_with_power_plants(substations, pwlines, iris, 
                                                       power_stations, pws_type_key="Filière")

Save the result:

In [8]:
france_substations.to_file(os.path.join(data_dir, "france_rte/RTE_substations"))